In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# 데이터 로드 (CSV 파일로 저장되어 있다고 가정)
item_data = pd.read_csv('../문항_매개변수.csv')
response_data = pd.read_csv('../응답_데이터.csv')

# 1. 문항 난이도 분포 히스토그램
plt.figure(figsize=(10, 6))
sns.histplot(item_data['난이도 (beta)'], bins=20, kde=True)
plt.title('문항 난이도 분포')
plt.xlabel('난이도')
plt.ylabel('빈도')
plt.show()

# 2. 문항 변별도 분포 히스토그램
plt.figure(figsize=(10, 6))
sns.histplot(item_data['변별도 (alpha)'], bins=20, kde=True)
plt.title('문항 변별도 분포')
plt.xlabel('변별도')
plt.ylabel('빈도')
plt.show()

# 3. 문항 난이도와 변별도의 산점도
plt.figure(figsize=(10, 6))
sns.scatterplot(data=item_data, x='난이도 (beta)', y='변별도 (alpha)')
plt.title('문항 난이도와 변별도의 관계')
plt.xlabel('난이도')
plt.ylabel('변별도')
plt.show()

# 4. 문항 반응 이론(IRT) 특성 곡선
# (이 부분은 IRT 라이브러리가 필요하므로 여기서는 생략합니다)

# 5. 문항별 정답률 막대 그래프
# 정답률 계산 (OX리스트를 이용)
correct_answers = response_data['OX리스트'].apply(lambda x: x.count('O'))
total_questions = response_data['OX리스트'].apply(len)
correct_rate = correct_answers / total_questions

plt.figure(figsize=(15, 6))
sns.barplot(x=item_data['문항 번호'], y=correct_rate)
plt.title('문항별 정답률')
plt.xlabel('문항 번호')
plt.ylabel('정답률')
plt.xticks(rotation=90)
plt.show()

# 6. 평가 영역별 평균 난이도/변별도 비교
avg_difficulty = item_data.groupby('평가영역')['난이도 (beta)'].mean()
avg_discrimination = item_data.groupby('평가영역')['변별도 (alpha)'].mean()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

avg_difficulty.plot(kind='bar', ax=ax1)
ax1.set_title('평가 영역별 평균 난이도')
ax1.set_ylabel('평균 난이도')

avg_discrimination.plot(kind='bar', ax=ax2)
ax2.set_title('평가 영역별 평균 변별도')
ax2.set_ylabel('평균 변별도')

plt.tight_layout()
plt.show()

# 추가 분석: 난이도와 변별도의 상관관계
correlation, p_value = stats.pearsonr(item_data['난이도 (beta)'], item_data['변별도 (alpha)'])
print(f"난이도와 변별도의 상관계수: {correlation:.2f}")
print(f"p-value: {p_value:.4f}")

# 문항 분석 요약 통계
summary_stats = item_data[['난이도 (beta)', '변별도 (alpha)']].describe()
print("\n문항 분석 요약 통계:")
print(summary_stats)

# 가장 어려운/쉬운 문항, 가장 높은/낮은 변별도를 가진 문항 식별
hardest_item = item_data.loc[item_data['난이도 (beta)'].idxmax()]
easiest_item = item_data.loc[item_data['난이도 (beta)'].idxmin()]
most_discriminating = item_data.loc[item_data['변별도 (alpha)'].idxmax()]
least_discriminating = item_data.loc[item_data['변별도 (alpha)'].idxmin()]

print("\n가장 어려운 문항:")
print(hardest_item)
print("\n가장 쉬운 문항:")
print(easiest_item)
print("\n가장 높은 변별도를 가진 문항:")
print(most_discriminating)
print("\n가장 낮은 변별도를 가진 문항:")
print(least_discriminating)